# Introduction

In this notebook, I go through how I worked to find a decent solution for this challenge using simple uncomplicated techniques. No machine learning, no fancy black-box models. Throw away your ARIMAs and Gradient Boosts. Think simple.

# Setup and Loading Data

In [1]:
%load_ext autoreload
# %autoreload 2
%matplotlib inline
%load_ext ipython_unittest

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_rows', 12)

In [2]:
train = pd.read_csv("train.csv", low_memory=False, 
                    parse_dates=['date'], index_col=['date'])
train.sort_index(inplace=True)

test = train.loc['2017-09':, :]
train = train.loc[:'2017-08', :]
# train = train.loc['2015-09':'2017-08', :]  #short
# test = train.loc['2017-09': '2017-10', :] #short


In [3]:
test.head()

,store,item,sales
date,,,
2017-09-01,2,5,29
2017-09-01,1,48,63
2017-09-01,2,45,138
2017-09-01,9,40,36
2017-09-01,8,17,48


In [4]:
def smape(A, F):
    A, F = A.astype(float), F.astype(float)

    return (100. / len(A)) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)  + np.finfo(float).eps))

### unit tests

In [5]:
%%unittest_testcase

def test_smape(self):
    A1, F1 = np.array([1, 2]), np.array([1.1, 2.2])
    expected_result1 = 100 * .5 * (.1 / 1.05 + .2 / 2.1)
    A2, F2 = np.array([1, 2]), np.array([3, 4])
    expected_result2 = 100 * .5 * (2 / 2 + 2. / 3)
    A3 = np.arange(101)
    F3 = np.arange(101)
    F3[100] = 101
    expected_result3 = (100. / 101) * (1 / (201. / 2))
    
    self.assertAlmostEqual(smape(A1,F1), expected_result1, places = 7)
    self.assertAlmostEqual(smape(A2,F2), expected_result2, places = 7)
    self.assertAlmostEqual(smape(A3,F3), expected_result3, places = 7)

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Exploring the data

In [6]:
# Expand dataframe with more useful columns
def expand_df(df):
    data = df.copy()
    data['day'] = data.index.day
    data['month'] = data.index.month
    data['year'] = data.index.year
    data['dayofweek'] = data.index.dayofweek
#     data['week_of_year']  = data.index.weekofyear
    return data

train = expand_df(train)
test = expand_df(test)



In [7]:
train_colums = ['store', 'item', 'month', 'year', 'dayofweek']
labels_column = 'sales'
data_x = train[train_colums].values
data_y = train[labels_column].values

data_test_x = test[train_colums].values
data_test_y = test[labels_column].values

In [8]:
regr_1 = DecisionTreeRegressor(max_depth=10)
regr_2 = DecisionTreeRegressor(max_depth=50)

regr_1.fit(data_x, data_y)
regr_2.fit(data_x, data_y)

DecisionTreeRegressor(criterion='mse', max_depth=50, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [9]:
y_1 = regr_1.predict(data_test_x)
y_2 = regr_2.predict(data_test_x)

In [11]:
print smape(y_1, data_test_y)
print smape(y_2, data_test_y)

29.54142495780605
13.584439225622969
